# Scryfall API ingestion for T2 Cards

## Imports

In [0]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *

## Ingestion

In [0]:
url = "https://api.scryfall.com/cards/search"

params = {"q": "format:standard", "order": "color", "unique": "cards", "dir": "asc"}

cards = []
while url:
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    cards.extend(data['data'])
    
    url = data.get('next_page', None)
    params = None  

def extract_mana_cost(card):
    if card.get("mana_cost"):
        return card["mana_cost"]
    elif "card_faces" in card and card["card_faces"]:
        return card["card_faces"][0].get("mana_cost", "")
    return ""

def extract_colors(card):
    if card.get("colors"):
        return card["colors"]
    elif "card_faces" in card and card["card_faces"]:
        colors = []
        for face in card["card_faces"]:
            if "colors" in face and face["colors"]:
                colors.extend(face["colors"])
        return list(set(colors))
    return []

def extract_oracle_text(card):
    if card.get("oracle_text"):
        return card["oracle_text"]
    elif "card_faces" in card and card["card_faces"]:
        texts = []
        for face in card["card_faces"]:
            if "oracle_text" in face and face["oracle_text"]:
                texts.append(face["oracle_text"])
        return " // ".join(texts) if texts else ""
    return ""

for c in cards:
    c["mana_cost"] = extract_mana_cost(c)
    c["colors"] = extract_colors(c)
    c["oracle_text"] = extract_oracle_text(c)

df_cards = pd.json_normalize(cards, sep='_')

cols = [
    'name', 'mana_cost', 'type_line', 'oracle_text',
    'power', 'toughness', 'loyalty', 'colors',
    'color_identity', 'set', 'set_name', 'scryfall_uri'
]
df_cards = df_cards[cols]

for col in ['colors', 'color_identity']:
    df_cards[col] = df_cards[col].apply(
        lambda x: ','.join(map(str, x)) if isinstance(x, list) else ''
    )

display(df_cards.head(20))
print(f"{len(df_cards)} Standard cards found")


## Saving df in Unity Catalog

In [0]:
spark_df = spark.createDataFrame(df_cards)

spark_df.write.format("delta").mode("overwrite").saveAsTable("mtg.bronze.t2_cards_ingestion")

print(f"Table successfully written")
